In [22]:
from nltk.corpus import wordnet 
import pronouncing
from itertools import product
import functools
import operator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
import string

In [108]:
stop = set(stopwords.words('english'))

INCLUDE_STOP = True
# https://www.geeksforgeeks.org/get-synonymsantonyms-nltk-wordnet-python/
def synonyms(word):
    synonyms = [word]
    
    if not INCLUDE_STOP and word in stop:
        return synonyms
    
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonyms.append(l.name()) 
                
    return synonyms

synonyms('slow')

['slow',
 'decelerate',
 'slow',
 'slow_down',
 'slow_up',
 'retard',
 'slow',
 'slow_down',
 'slow_up',
 'slack',
 'slacken',
 'slow',
 'slow_down',
 'slow_up',
 'slow',
 'slow',
 'dense',
 'dim',
 'dull',
 'dumb',
 'obtuse',
 'slow',
 'slow',
 'boring',
 'deadening',
 'dull',
 'ho-hum',
 'irksome',
 'slow',
 'tedious',
 'tiresome',
 'wearisome',
 'dull',
 'slow',
 'sluggish',
 'slowly',
 'slow',
 'easy',
 'tardily',
 'behind',
 'slow']

In [44]:
def meter(phrase):
    phrase = phrase.translate(str.maketrans({a:None for a in string.punctuation})) # strip punctuation
    syllables = 0 
    stresses = []
    for word in word_tokenize(phrase):
#         try:
        phonemes = pronouncing.phones_for_word(word)[0]
        syllables += pronouncing.syllable_count(phonemes)
        stresses.append(list(map(int, pronouncing.stresses(phonemes))))
#         except IndexError:
# #             print(f'\tError: word without phoneme found: "{word}"')
#             syllables += 1
#             stresses.append([1])

    return (syllables, stresses)

In [45]:
meter('that was amazing')

(5, [[1], [1], [0, 1, 0]])

In [46]:
def check_iambic(line):
    syllables, stresses = meter(line)
    
    if syllables is not 10:
        return False
    
    expected_next = False # not stressed
    
    for stress in stresses:
        if len(stress) == 1:
            expected_next = not expected_next
            continue # one syllable can be stressed or not stressed
        
        for syllable in stress:
            if syllable == 2:
                expected_next = not expected_next
                continue # lightly stressed, can be either
                
            if (syllable == 1) == expected_next:
                expected_next = not expected_next
                continue
            else:
                return False
    return True

In [47]:
print(check_iambic('How do I love thee? Let me count the ways.'))
print(check_iambic('Is this smarter than simply syllables?'))

True
False


In [49]:
test_poem = \
'''How do I love thee? Let me count the ways.
I love thee to the depth and breadth and height
My soul can reach, when feeling out of sight
For the ends of being and ideal grace.
I love thee to the level of every day's
Most quiet need, by sun and candle-light.
I love thee freely, as men strive for right.
I love thee purely, as they turn from praise.
I love thee with the passion put to use
In my old griefs, and with my childhood's faith.
I love thee with a love I seemed to lose
With my lost saints. I love thee with the breath,
Smiles, tears, of all my life; and, if God choose,
I shall but love thee better after death.
'''.split('\n')
for line in test_poem:
    try:
        if not check_iambic(line):
            print(line)
    except:
        pass

For the ends of being and ideal grace.
I love thee to the level of every day's



In [106]:
def make_iambic(line):
    words = line.split(' ') # word_tokenize(line)
    word_synonyms = list(map(synonyms, words))
    
    for i, syns in enumerate(word_synonyms):
        if len(syns) == 0:
            word_synonyms[i] = [words[i]]
            
    iambic = []
    # https://stackoverflow.com/questions/32074543/how-to-get-the-length-of-an-itertools-product
    n_possible = functools.reduce(operator.mul, map(len, word_synonyms), 1)
    for possible in tqdm(product(*word_synonyms), total=n_possible):
        potential = ' '.join(possible)
        try:
            if check_iambic(potential):
                iambic.append(potential)
        except IndexError: # can't find pronunciation
            continue
    return iambic

In [109]:
INCLUDE_STOP = True
make_iambic('For the ends of being and ideal grace.')

HBox(children=(IntProgress(value=0, max=15210), HTML(value='')))

['For the ends of existence and saint grace.',
 'For the ends of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the last of existence and saint grace.',
 'For the last of embody and saint grace.',
 'For the goal of existence and saint grace.',
 'For the goal of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the end of existence and saint grace.',
 'For the end of embody and saint grace.',
 'For the destruction of be and saint grace.',
 'For the destruction of be and saint grace.',
 'For the destruction of be and saint grace.',
 'For the destruction of be and saint grace.',
 'For

In [61]:
corpus = word_tokenize(open('sermon_mount_NIV.txt').read())

In [65]:
def find_iambic(corpus):
    

'in'